In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold,GridSearchCV


In [ ]:
data=pd.read_csv("gas_turbines.csv")

In [ ]:
data

In [ ]:
data.isna().sum()

In [ ]:
data.info()

In [ ]:
data.corr()

In [ ]:
data.describe()

In [ ]:
data.plot()

In [ ]:
data.columns

In [ ]:
#Ambient temperature -->(AT) # information given.
#Ambient pressure--> (AP) 
#Ambient humidity---> (AH) 
#Air filter difference pressure---> (AFDP)
#Gas turbine exhaust pressure---> (GTEP) 
#Turbine inlet temperature----> (TIT)
#Turbine after temperature---> (TAT) 
#Compressor discharge pressure---> (CDP) 
#Turbine energy yield---> (TEY)(target variable)
#Carbon monoxide----> (CO) 
#Nitrogen oxides---> (NOX)


In [ ]:
import seaborn as sns
sns.pairplot(data=data,hue="TEY")

In [ ]:
import matplotlib.pyplot as plt
numerical_features=[feature for feature in data.columns if data[feature].dtypes != 'O']
for feat in numerical_features:
    skew = data[feat].skew()
    sns.distplot(data[feat], kde= False, label='Skew = %.3f' %(skew), bins=30)
    plt.legend(loc='best')
    plt.show()
   
   

In [ ]:
a=StandardScaler()
b=a.fit_transform(data)
data1=pd.DataFrame(b,columns=("AT",	"AP",	"AH",	"AFDP",	"GTEP",	"TIT",	"TAT",	"TEY",	"CDP"	,"CO",	"NOX"))

In [ ]:
data1

In [ ]:
X=data1.drop(columns="TEY")
Y=data1["TEY"]

In [ ]:
X

In [ ]:
Y

In [ ]:
pip install tensorflow

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense

from keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.keras import initializers
import warnings
warnings.filterwarnings('ignore')
from tensorflow.keras.optimizers import Adam

In [ ]:
def model1():
  model =Sequential()
  model.add(Dense(10,input_dim=10,activation='relu'))
  model.add(Dense(8,activation='relu'))
  model.add(Dense(1,activation='relu'))

  adam=Adam(lr=0.1)
  model.compile(loss="mean_squared_error",optimizer=adam,metrics=["mean_squared_error"])
  return model

In [ ]:
model=KerasRegressor(build_fn=model1)
# Define the grid search parameters
batch_size = [10]
epochs = [5]
# Make a dictionary of the grid search parameters
param_grid = dict(batch_size = batch_size,epochs = epochs)
# Build and fit the GridSearchCV
grid = GridSearchCV(estimator = model,param_grid = param_grid,cv = KFold())


In [ ]:
grid.fit(X,Y)

In [ ]:
print("best:{},using {}".format(grid.best_score_,grid.best_params_))

In [ ]:
print("best:{},using {}".format(grid.best_score_,grid.best_params_))
mean=grid.cv_results_['mean_test_score']
std=grid.cv_results_['std_test_score']
params = grid.cv_results_['params']
for mean,std,param in zip(mean,std,params):
  print("{},{} with {}".format(mean,std,params))


In [ ]:
#tunning the hyperparmetern learning_rate,dropout_rate

In [ ]:

from keras.layers import Dropout
def model2(learning_rate,dropout_rate):
  model=Sequential()
  model.add(Dense(10,input_dim=10,activation="relu"))
  model.add(Dropout(dropout_rate))
  model.add(Dense(8,activation="relu"))
  model.add(Dropout(dropout_rate))
  model.add(Dense(1,activation="sigmoid"))

  adam=Adam(lr=learning_rate)
  model.compile(loss="mean_squared_error",optimizer=adam,metrics=["mean_squared_error"])
  return model

In [ ]:
model_2=KerasRegressor(build_fn=model2,batch_size=40,epochs=50)
learning_rate = [0.001,0.01]
dropout_rate=[0.2,0.1]

param=dict(learning_rate=learning_rate,dropout_rate=dropout_rate)
grid=GridSearchCV(estimator=model_2,param_grid=param,cv=KFold(),verbose=10)
grid_results=grid.fit(X,Y)


In [ ]:
print("best:{},using {}".format(grid_results.best_score_,grid_result.best_params_))
mean=grid_results.cv_results_['mean_test_score']
std=grid_results.cv_results_['std_test_score']
params = grid_results.cv_results_['params']
for mean,std,param in zip(mean,std,params):
  print("{},{} with {}".format(mean,std,param))


In [ ]:
#tunning the hyperparamter initializer,activation_function

In [ ]:
from keras.layers import LeakyReLU

In [ ]:
from tensorflow.keras import layers
def model_3(init,activation_function):
  model=Sequential()
  model.add(layers.Dense(10,input_dim=10,kernel_initializer=init,activation="activation_function"))
  model.add(Dropout(0.2))
  model.add(layers.Dense(8,activation="activation_function"))
  model.add(Dropout(0.2))
  model.add(layers.Dense(1,activation="activation_function"))

  adam=Adam(lr=0.3)
  model.compile(optimizer=adam,loss="mean_squared_error",metrics=["mean_squared_error"])
  return model

In [ ]:

model=KerasRegressor(build_fn=model_3,batch_size=20,epochs=50)

activation_function = ['LeakyReLU']
init = ['uniform','normal','zero']

param_grids = dict(init=init,activation_function=activation_function)


grid = GridSearchCV(estimator = model,param_grid = param_grids,cv = KFold(),verbose = 10)
grid_result = grid.fit(X,Y)


In [ ]:

print('Best : {}, using {}'.format(grid_result.best_score_,grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
  print('{},{} with: {}'.format(mean, stdev, param))

In [ ]:
#tunning the model by neuron in network

In [ ]:
# Defining the model

def create_model(neuron1,neuron2):
    model = Sequential()
    model.add(Dense(neuron1,input_dim=10,kernel_initializer = 'uniform',activation = 'relu'))
    model.add(Dropout(0.1))
    model.add(Dense(neuron2,input_dim=neuron1,kernel_initializer = 'uniform',activation = 'relu'))
    model.add(Dropout(0.1))
    model.add(Dense(1,activation = 'sigmoid'))
    
    adam = Adam(lr = 0.001)
    model.compile(loss = 'mean_squared_error',optimizer = adam,metrics = ['mean_squared_error'])
    return model

In [ ]:



model = KerasRegressor(build_fn = create_model,verbose = 0,batch_size = 40,epochs = 10)


neuron1 = [4,8,16]
neuron2 = [2,4,8]



param_grids = dict(neuron1 = neuron1,neuron2 = neuron2)


grid = GridSearchCV(estimator = model,param_grid = param_grids,cv = KFold(),verbose = 10)
grid_result = grid.fit(X,Y)

In [ ]:
print('Best : {}, using {}'.format(grid_result.best_score_,grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
  print('{},{} with: {}'.format(mean, stdev, param))

In [ ]:
#finding the accuracy

In [ ]:
from sklearn.metrics import classification_report, accuracy_score

In [ ]:


def create_model():
    model = Sequential()
    model.add(Dense(16,input_dim = 10,kernel_initializer = 'uniform',activation = 'tanh'))
    model.add(Dropout(0.1))
    model.add(Dense(4,input_dim = 16,kernel_initializer = 'uniform',activation = 'tanh'))
    model.add(Dropout(0.1))
    model.add(Dense(1,activation = 'sigmoid'))
    
    adam = Adam(lr=0.01)
    model.compile(loss = 'binary_crossentropy',optimizer = adam,metrics = ['accuracy'])
    return model

model = KerasRegressor(build_fn = create_model,verbose = 0,batch_size = 40,epochs = 10)



model.fit(X,Y)



In [ ]:
from sklearn.model_selection import cross_val_score
results=cross_val_score(model,X,Y,cv=KFold())


In [ ]:
results